# étude de l'article 2

In [10]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation_arcticle_1 import *
from src.modelisation_arcticle_2 import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [11]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

In [12]:
df_modelisation = read_and_prepare_df_of_the_model('01. output/df_freqs_speaker_word.csv')

In [13]:
df_modelisation['m_it'] = df_modelisation.groupby(by=['Speaker', 'party'])['value'].transform('sum')

In [14]:
# On Calcule les termes ds le sigma 
df_modelisation['q_hat_it'] = df_modelisation['value']/df_modelisation['m_it']

In [15]:
df_modelisation_Lab = df_modelisation.loc[df_modelisation['party'] == 'Lab']
df_modelisation_Con = df_modelisation.loc[df_modelisation['party'] == 'Con']

## plug-in estimator (biased)

In [16]:
# je suis pas du tt sur je crosi c'est faux
df_modelisation_Lab_plugin = plugin_estimator(df_modelisation_Lab)
df_modelisation_Con_plugin = plugin_estimator(df_modelisation_Con)

In [17]:
df_modelisation_Con

,Speaker,party,variable,value,m_it,q_hat_it,Somme_m_it,Somme_c_it,q_hat_Lab_jt
2,George Hollingbery,Con,abil,3,1948,0.001540,974000,1948,0.002
3,George Freeman,Con,abil,12,3886,0.003088,1943000,3886,0.002
4,George Eustice,Con,abil,13,9312,0.001396,4656000,9312,0.002
6,Geoffrey Cox,Con,abil,0,155,0.000000,77500,155,0.002
7,Geoffrey Clifton-Brown,Con,abil,5,3940,0.001269,1970000,3940,0.002
...,...,...,...,...,...,...,...,...,...
434987,Mary Robinson,Con,â£,10,1211,0.008258,605500,1211,0.002
434988,Mary Macleod,Con,â£,2,648,0.003086,324000,648,0.002
434992,Martin Vickers,Con,â£,3,3692,0.000813,1846000,3692,0.002
434998,Matt Hancock,Con,â£,40,15426,0.002593,7713000,15426,0.002


## leave-out estimator (not biased)

In [8]:
lox_de_lox = df_modelisation_Con[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
df_modelisation_Con = pd.merge(
    df_modelisation_Con,
    lox_de_lox,
    how='left',
    on=['variable']
)

In [9]:
df_modelisation_Con

,Speaker,party,variable,value,m_it,q_hat_it,Somme_m_it,Somme_c_it_x,q_hat_Lab_jt,Somme_c_it_y
0,George Hollingbery,Con,abil,3,1948,0.001540,974000,1948,0.002,1921
1,George Freeman,Con,abil,12,3886,0.003088,1943000,3886,0.002,1921
2,George Eustice,Con,abil,13,9312,0.001396,4656000,9312,0.002,1921
3,Geoffrey Cox,Con,abil,0,155,0.000000,77500,155,0.002,1921
4,Geoffrey Clifton-Brown,Con,abil,5,3940,0.001269,1970000,3940,0.002,1921
...,...,...,...,...,...,...,...,...,...,...
214995,Mary Robinson,Con,â£,10,1211,0.008258,605500,1211,0.002,5831
214996,Mary Macleod,Con,â£,2,648,0.003086,324000,648,0.002,5831
214997,Martin Vickers,Con,â£,3,3692,0.000813,1846000,3692,0.002,5831
214998,Matt Hancock,Con,â£,40,15426,0.002593,7713000,15426,0.002,5831


In [12]:
lox_de_lox = df_modelisation_Con[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
df_modelisation_Con = pd.merge(
    df_modelisation_Con,
    lox_de_lox,
    how='left',
    on=['variable']
)
df_modelisation_Con['Somme_c_it'] = df_modelisation_Con['Somme_c_it'] - df_modelisation_Con['value']
lox_de_lox = df_modelisation.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
df_modelisation_Con = pd.merge(
    df_modelisation_Con,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
df_modelisation_Con['Somme_m_it'] = df_modelisation_Con['Somme_m_it'] - df_modelisation_Con['m_it']
df_modelisation_Con['q_hat_Con_-ijt'] = df_modelisation_Con['Somme_c_it'] / df_modelisation_Con['Somme_m_it']

In [13]:
df_modelisation_Con

,Speaker,party,variable,value,m_it,q_hat_it,Somme_c_it,Somme_m_it,q_hat_Con_-ijt
0,George Hollingbery,Con,abil,3,1948,0.001540,1918,2495823,0.000768
1,George Freeman,Con,abil,12,3886,0.003088,1909,2493885,0.000765
2,George Eustice,Con,abil,13,9312,0.001396,1908,2488459,0.000767
3,Geoffrey Cox,Con,abil,0,155,0.000000,1921,2497616,0.000769
4,Geoffrey Clifton-Brown,Con,abil,5,3940,0.001269,1916,2493831,0.000768
...,...,...,...,...,...,...,...,...,...
214995,Mary Robinson,Con,â£,10,1211,0.008258,5821,2496560,0.002332
214996,Mary Macleod,Con,â£,2,648,0.003086,5829,2497123,0.002334
214997,Martin Vickers,Con,â£,3,3692,0.000813,5828,2494079,0.002337
214998,Matt Hancock,Con,â£,40,15426,0.002593,5791,2482345,0.002333


In [14]:
lox_de_lox = df_modelisation_Lab[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
df_modelisation_Lab = pd.merge(
    df_modelisation_Lab,
    lox_de_lox,
    how='left',
    on=['variable']
)
df_modelisation_Lab['Somme_c_it'] = df_modelisation_Lab['Somme_c_it'] - df_modelisation_Lab['value']
lox_de_lox = df_modelisation.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
df_modelisation_Lab = pd.merge(
    df_modelisation_Lab,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
df_modelisation_Lab['Somme_m_it'] = df_modelisation_Lab['Somme_m_it'] - df_modelisation_Lab['m_it']
df_modelisation_Lab['q_hat_Lab_-ijt'] = df_modelisation_Lab['Somme_c_it'] / df_modelisation_Lab['Somme_m_it']

In [15]:
df_modelisation_Lab_intial = df_modelisation_Lab.drop(columns=['q_hat_Lab_-ijt'])
df_modelisation_Lab_intial['Somme_m_it'] = df_modelisation_Lab_intial['Somme_m_it'] + df_modelisation_Lab_intial['m_it']
df_modelisation_Lab_intial['Somme_c_it'] = df_modelisation_Lab_intial['Somme_c_it'] + df_modelisation_Lab_intial['value']
df_modelisation_Lab_intial['q_hat_Lab_jt'] = df_modelisation_Lab_intial['Somme_c_it'] / df_modelisation_Lab_intial['Somme_m_it']
df_modelisation_Lab_intial = df_modelisation_Lab_intial[['variable', 'q_hat_Lab_jt']]
df_modelisation_Lab_intial.drop_duplicates(subset=['variable'], inplace=True)

In [16]:
# La je fais mon merge 
df_modelisation_Con = pd.merge(
    df_modelisation_Con,
    df_modelisation_Lab_intial,
    how='left',
    on=['variable']
)
df_modelisation_Con['rho_hat_-ijt'] = (df_modelisation_Con['q_hat_Con_-ijt']) / (df_modelisation_Con['q_hat_Con_-ijt'] + df_modelisation_Con['q_hat_Lab_jt'])
df_modelisation_Con['produit'] = df_modelisation_Con['q_hat_it'] * df_modelisation_Con['rho_hat_-ijt']
nb_Con = len(df_modelisation_Con['Speaker'].unique())
res_Con = df_modelisation_Con[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Con['moitie_Con_pi'] = res_Con['produit']*(1/2)*(1/nb_Con)
res_Con.drop(columns=['produit'], inplace=True)

## je refais pour les Labors 

In [17]:
df_modelisation_Con_intial = df_modelisation_Con.drop(columns=['q_hat_Con_-ijt'])
df_modelisation_Con_intial['Somme_m_it'] = df_modelisation_Con_intial['Somme_m_it'] + df_modelisation_Con_intial['m_it']
df_modelisation_Con_intial['Somme_c_it'] = df_modelisation_Con_intial['Somme_c_it'] + df_modelisation_Con_intial['value']
df_modelisation_Con_intial['q_hat_Con_jt'] = df_modelisation_Con_intial['Somme_c_it'] / df_modelisation_Con_intial['Somme_m_it']
df_modelisation_Con_intial = df_modelisation_Con_intial[['variable', 'q_hat_Con_jt']]
df_modelisation_Con_intial.drop_duplicates(subset=['variable'], inplace=True)

In [18]:
#  La je fais mon merge 
df_modelisation_Lab = pd.merge(
    df_modelisation_Lab,
    df_modelisation_Lab_intial,
    how='left',
    on=['variable']
)
df_modelisation_Lab['rho_hat_-ijt'] = (df_modelisation_Lab['q_hat_Lab_-ijt']) / (df_modelisation_Lab['q_hat_Lab_-ijt'] + df_modelisation_Lab['q_hat_Lab_jt'])
df_modelisation_Lab['produit'] = df_modelisation_Lab['q_hat_it'] * df_modelisation_Lab['rho_hat_-ijt']
nb_Lab = len(df_modelisation_Lab['Speaker'].unique())
res_Lab = df_modelisation_Lab[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Lab['moitie_Lab_pi'] = res_Lab['produit']*(1/2)*(1/nb_Lab)
res_Lab.drop(columns=['produit'], inplace=True)

In [19]:
res = pd.merge(
    res_Con,
    res_Lab,
    how='left',
    on=['variable'],
)

In [20]:
res['pi'] = res['moitie_Con_pi'] + res['moitie_Lab_pi']

In [21]:
res

,variable,moitie_Con_pi,moitie_Lab_pi,pi
0,abil,0.000364,0.000237,0.000601
1,abl,0.002495,0.001939,0.004434
2,absolut,0.000720,0.000358,0.001078
3,accept,0.000479,0.000522,0.001001
4,achiev,0.000389,0.000286,0.000676
...,...,...,...,...
495,withdraw,0.000137,0.000106,0.000243
496,word,0.000279,0.000383,0.000662
497,write,0.000090,0.000091,0.000180
498,yes,0.000303,0.000296,0.000599


In [22]:
res['pi'].max()

0.08763558698043365

## Test sur un speaker

In [23]:
pi_speaker_Con = df_modelisation_Con[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()


In [24]:
# Pour les Con
pi_speaker_Con = df_modelisation_Con[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()
pi_speaker_Con['party'] = 'Con'

# Pour les Lab
pi_speaker_Lab = df_modelisation_Lab[['Speaker','produit']].groupby(by=['Speaker']).sum().reset_index()
pi_speaker_Lab['party'] = 'Lab'


In [25]:
pi_speaker = pd.concat([pi_speaker_Con, pi_speaker_Lab])
pi_speaker.rename(columns={'produit':'speaker_pi'},inplace=True)
pi_speaker = pi_speaker.loc[pi_speaker['speaker_pi'] > 0]

In [27]:
import plotly.express as px
fig = px.scatter(pi_speaker, y="party", x="speaker_pi", color="party", symbol="party")
fig.update_traces(marker_size=10)
fig.show()

## avec les articles 

In [32]:
df_Guardian = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

Il faut d'abord effectuer un traitement similaire df_modelisation articles des journdf_modelisation

In [1]:
def treat_1(df, party):
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [2]:
def treat_2(df_1,df_2):
  df = pd.concat([df_1, df_2])
  df.dropna().reset_index(drop=True)
  return df

In [3]:
def treat_3(df):
  df['text'] = df['text'].apply(clean, args=(stop_words,))
  return df 

In [4]:
df_Guardian = treat_1(df_Guardian, 'Guardian')

NameError: name 'df_Guardian' is not defined

In [5]:
df_DE = treat_1(df_DE, 'DE')

NameError: name 'df_DE' is not defined

In [ ]:
df_newspaper = treat_2(df_Guardian, df_DE)

In [ ]:
df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
df_newspaper = df_newspaper.head(1000)

In [ ]:
df_newspaper = treat_3(df_newspaper)

In [ ]:
df_newspaper_treated = df_newspaper

In [ ]:
df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()

In [ ]:
df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')

In [ ]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

In [ ]:
list_of_words = selected_words(df_freqs)

In [ ]:
df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)

In [ ]:
df_newspaper_treated['Unnamed: 0'] = 0

In [ ]:
LO_treatment(df_newspaper_treated)

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment(df_newspaper_treated)

In [ ]:
treat_fonction(df_Guardian, df_DE)

# **Évolution de la polarisation dans le temps**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
def treat_1_date(df, party, year):
  df = df.loc[df['year'] == year]
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  print('SORTIE')
  return LO_treatment(df_newspaper_treated)

In [ ]:
df_1 = treat_1_date(df_Guardian, 'Guardian', 2020)
df_2 = treat_1_date(df_DE, 'DE', 2020)

In [ ]:
treat_fonction(df_1, df_2)

In [ ]:
evolution_of_pi = []

In [ ]:
for year in range(2010, 2023 ):
  df_a = treat_1_date(df_Guardian, 'Guardian', year)
  df_b = treat_1_date(df_DE, 'DE', year)
  pi = treat_fonction(df_a, df_b)
  evolution_of_pi.append(pi)
  print(evolution_of_pi, 'BOUCLE')

In [ ]:
evolution_of_pi

In [ ]:
plt.plot([i for i in range(2010, 2023)], evolution_of_pi)
plt.show()

# **Partisanship des mots**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
def treat_table(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment_table(df_newspaper_treated)

In [ ]:
table_Con, table_Lab = treat_table(df_Guardian, df_DE)

In [ ]:
table_Con.dropna().reset_index(drop=True)
table_Lab.dropna().reset_index(drop=True)